TODO CHECK SIZES

Introduce Ax=b. A is square
Equivalence between matrix-vector multiplication and linear systems of equations
Basis, Nullspace of A, describes why there are no solutions for some Ax=b
Fundamental theorem - null space generating set is independent of spanning set?
Two cases: Invertible (Ax=b has a unique solution), non-invertible (Ax=b has infinitely many or no solutions)
Optional (?): Dedicated time on linear (in)dependence

Now, Regression → Ax + b = y. Frame this as Ax=y-b with a non-square A
More columns than rows → no solutions, in general.

Informal: Solve for Normal equation
Interpret terms of normal equation in terms of nullspace (X^TX)
Edge case: More columns than rows → infinite solutions
Optional: Collinearity / rank-deficiency
Optional: Note anything about intercept?
Optional: Ill-conditioned matrices


# Solving a system
In this notebook, we focus on solving a system of equations. This is ultimately related to regression, but we intentionally focus on this simple presentation to stress some of the concepts around when regression does and does not have solutions.

Let's start with this simple system of equations and try to solve it.

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

1. We can solve for $x_1$ by subtracting the line 3 from line 1, so $x_1=1$.
2. We can plug that into line 2, so our system of equations is now

$$
\begin{align}
x_1 &= 1 \\
- x_2 + 2 x_3 &= 1 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

3. We add line 3 to line 2. That leaves us with $3 x_3 = 3$, so $x_3 = 1$.
4. That means $x_2=1$! So our solution $\mathbf{x} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$.

We just followed an ad-hoc algorithm to solve this matrix, but there are more systematic ways to solve it by 1) systematically repeating simplification steps (called Gaussian eliminiation) until 2) each row reaches a _canonical_ form (reduced row-echelon form). This approach formalizes the above algorithm into a xxxxxx, but for the purpose of this tutorial, we won't cover it any further xxxxx.

What do these systems of equations have in common with linear regression? First, these simple cases will help us understand the geometry of linear regression. Second, they also have the same solutions, in the general case. Something about collinearity?

# A more problematic system

Now, let's examine a different linear system to see some of the different kinds of solutions they can have.

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
2 x_1 + 3 x_3 &= 1 \\
\end{align}
$$

If you add line 2 to line 1, you get $2 x_1 + 3 x_3 = 5$, which is a contradiction! So, there's no way to solve this system of equations, making them inconsistent. But what happens if we change this to avoid that contradiction? If we only modify the last equation to avoid the contradiction, we get this system (change in bold)

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
2 x_1 + 3 x_3 &= \textbf{5} \\
\end{align}
$$

From this, we can reduce to two equations, obtaining the first by subtracting line 2 from line 1

$$
\begin{align}
2 x_2 - x_3 &= 1 \\
2 x_1 + 3 x_3 &= 5 \\
\end{align}
$$

Any $\textbf{x}$ that satisfies these equations is a solution, like $\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$ or $\begin{bmatrix} -2 & 2 & 3 \end{bmatrix}$. In fact, there are infinitely many solutions!

So, what's special about this system of equations? In our example in the previous section, the system of equations seemed to have a single unique solution. In this example, depending on the $\textbf{b}$ on the right, its possible to have either zero or infinite solutions.

# The many forms of matrix multiplication

In this section, we write out some notation to relate the above systems of equations with the matrix equation $\textbf{A}\mathbf{x}=\mathbf{b}$.

First, some notation about matrices. Consider a matrix $\textbf{A}$ of size $n \times m$, which means it has $n$ rows and $m$ columns.
We can write it out in terms of the individual entries of the matrix. The entry at row $i$ and column $j$ is $a_{ij}$, so the matrix is

$$
\textbf{A} = \begin{bmatrix}
a_{11} & a_{12} & \ldots & a_{1m} \\
a_{21} & a_{22} & \ldots & a_{2m} \\
\ldots & \ldots & \ldots & \ldots \\
a_{n1} & a_{n2} & \ldots & a_{nm} \\
\end{bmatrix}
$$

Another way we can write it is in terms of the columns. The $i^{th}$ column is $\textbf{a}_i$, so the matrix is

$$
\textbf{A} = \begin{pmatrix} \textbf{a}_1 & \textbf{a}_2 & \ldots & \textbf{a}_m \end{pmatrix}
$$

So, if we go back to our above example equations, we can rewrite them using some of these notational ideas. Here's the system of equations from the first section:

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

Each of these equations can correspond to a dot-product between $\textbf{x}$ and a vector with the factors (e.g., $\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$ for the first row). So, we can write this as a matrix equation

$$
\begin{bmatrix}
1 & 1 & 1 \\
1 & -1 & 2 \\
0 & 1 & 1 \\
\end{bmatrix} \mathbf{x} = \begin{bmatrix} 3 \\ 2 \\ 2 \end{bmatrix}
$$

Another different approach is to write it as a vector equation, using the columns of the above matrix.
$$
\begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix} x_1 +
\begin{bmatrix} 1 \\ -1 \\ 1 \end{bmatrix} x_2 +
\begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} x_3
= \begin{bmatrix} 3 \\ 2 \\ 2 \end{bmatrix}
$$

All three of these representations are equivalent, and are all solved by $\textbf{x} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$. We will change between them to emphasize different points below.

# Explaining the contradiction

So, to be specific, when we were solving system of equations above, we were equivalently trying to find a solution $\textbf{x}$ that satisfies the
the equation $\textbf{A}\textbf{x}=\textbf{b}$, given a specific matrix $\textbf{A}$ and vector $\textbf{b}$. From our above examples, a natural question followup question takes a step back and tries to generalize to arbitrary $\textbf{b}$ by asking: Given a matrix $\textbf{A}$, can we generalize characterize the $\textbf{b}$ that will have a solution?

Let's look at our problematic example above, now in matrix form:
$$
\textbf{A} = \begin{bmatrix}
1 & 1 & 1 \\
1 & -1 & 2 \\
2 & 0 & 3 \\
\end{bmatrix}
$$

We can rewrite the equation $\textbf{A}\textbf{x}=\textbf{b}$ in a vectorized form as follows

$$
\textbf{a}_1 x_1 + \textbf{a}_2 x_2 + \ldots + \textbf{a}_m x_m = \textbf{b}
$$

Or, more specifically for our current example
$$
\begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} x_1 +
\begin{bmatrix} 1 \\ -1 \\ 0 \end{bmatrix} x_2 +
\begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix} x_3
= \textbf{b}
$$

This makes it clear that $\textbf{b}$ is a **linear combination** of the columns of $\textbf{A}$. A linear combination is a weighted sum of vectors. In this case, the weights are the elements of $\textbf{x}$, and the vectors that we combine are the columns of $\textbf{A}$. A related concept is the **span** of a generating set of vectors, which is the set of all possible linear combinations of the vectors. One final concept: The column space of a matrix is the span of its columns. So, we can rephrase the above as: vectors $\textbf{b}$ with a solution must be in the column space of $\textbf{A}$.

So that explains part of what we saw above: we reached a contradiction with $\textbf{b} = \begin{bmatrix} 3 & 2 & 1 \end{bmatrix}$ because it cannot be expressed as a linear combination of the columns of $\textbf{A}$.

TODO Add in discussion of linear dependence, minimal generating set, basis

# Infinite solutions?

In order to explain the infinite number of solutions above, we'll have to introduce the **null space**, which are the solutions $\textbf{x'}$ so that $\textbf{A}\textbf{x'} = \textbf{0}$. Every matrix has the trivial solution $\textbf{x'}=\textbf{0}$, but only some matrices have other non-trivial solutions. The null space might seem like a strange concept, but it's useful! Given known solutions $\textbf{x}$ and $\textbf{x'}$, you can construct another solution to $\textbf{A}\textbf{x}=\textbf{b}$ by adding these equations together, since

$$
\begin{align}
\textbf{A}\textbf{x} + \textbf{A}\textbf{x'} &= \textbf{b} + \textbf{0} \\
\textbf{A}(\textbf{x} + \textbf{x'}) &= \textbf{b} \\
\end{align}
$$

So that means $\textbf{x}+\textbf{x'}$ is another solution. Since the null space can be useful for understanding cases with infinite solutions, let's see if we can solve $\textbf{A}\textbf{x} =\textbf{0}$ for our above matrices. We'll work with our problematic example in equation form

$$
\begin{align}
x_1 + x_2 + x_3 &= 0 \\
x_1 - x_2 + 2 x_3 &= 0 \\
2 x_1 + 3 x_3 &= 0 \\
\end{align}
$$

If we subtract line 2 from line 1 like we did above, we get the following constraints

$$
\begin{align}
2 x_2 - x_3 &= 0 \\
2 x_1 + 3 x_3 &= 0 \\
\end{align}
$$

These constraints are satisfied by solutions like $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$ or $\begin{bmatrix} -6 & 2 & 4 \end{bmatrix}$. If we look at the above case of infinite solutions ($\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$, $\begin{bmatrix} -2 & 2 & 3 \end{bmatrix}$), you'll notice that their difference is actually $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$. So, it looks like the null space is the span of a set containing the vector $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$. This explains the XXX TKTK

Of note: We were able to use the same process to solve this equation as above in order to get constraints. This generally TKTK and is a property that can be used to efficiently reuse TKTK. Or something about inverse?

# A fundamental theorem

TODO
TODO
TODO

- Introduce linear dependence, using columns of problematic above. Note that an element of the null set is a recipe for demonstrating linear dependence.

- Introduce dimensionality of set of vectors as the size of a minimal set, which is linearly independent.

- Now mention fundamental theorem, without proof, that for a square matrix $n \times n$, the $dim(col(\textbf{A})) + dim(null(\textbf{A})) = n$

- Basic idea is that all data is being mapped onto 0, or onto something in the output. Zero-sum!

- Introduce rank of matrix too.

# The unproblematic matrix

So, going back to our very first example, is there something special about the matrix for that system of equations? Let's try to compute its null space.

$$
\begin{align}
x_1 + x_2 + x_3 &= 0 \\
x_1 - x_2 + 2 x_3 &= 0 \\
x_2 + x_3 &= 0 \\
\end{align}
$$

We follow the same sequence of simplification steps as above, so

1. Subtract line 3 from line 1, so $x_1=0$.
2. Plug $x_1=0$ into line 2 and add line 3, so $3 x_3 = 0$.
3. Plug $x_3=0$ into line 3, so $x_2=0$.

So, the only solution to $\textbf{A}{x'}=\textbf{0}$ for this matrix is $\textbf{0}$. By the above fundamental theorem, that means $dim(col(\textbf{A})) = n$, so all data can be "reached" TKTK clarify this.


# Invertible matrices

Optional?

- Invertibility requires a trivial null space. If it were non-trivial, you couldn't go backwards from $0$.

- If your matrix is invertible, you can write a simpler way to solve the above

$$
\begin{align}
\textbf{A} \textbf{x} &= \textbf{b} \\
\textbf{x} &= \textbf{A}^{-1} \textbf{b} \\
\end{align}
$$

However, in practice, it's computationally more expensive and less accurate to use this process, so you'd prefer to use a dedicated linear solver like above.

# Regression

Keeping our symbols as above, linear regression is formulated as finding a minimum-error weight vector $\textbf{x}^*$ of size $m$ that can weight the columns of a feature matrix $\textbf{A}$ of size $n \times m$ to reproduce an expected output $\textbf{b}$ of size $m$. This matrix $\textbf{A}$ is a design matrix, with $n$ observations as rows and $m$ features as columns. Formally,

$$
\textbf{x}^* = \argmax_x \|\textbf{A}\textbf{x} - \textbf{b}\|_2
$$

By taking a derivative to solve for the maximum of this expression, we get the following equation, which resembles the simple approach to solving a linear system of equations we used above.

$$
\textbf{x}^* = (\textbf{A}^T\textbf{A})^{-1}\textbf{A}^T\textbf{b}
$$

This crucially depends on being able to take the inverse of $\textbf{A}^T\textbf{A}$, which is of size $m \times m$. For rectangular matrices, the maximum possible rank is $min(m, n)$, so that divides our xxxx

## When $n < m$, or more features than observations

When $n < m$, the matrix rank can be at most $n$, so $rank(\textbf{A}) <= n$. That means that $rank(\textbf{A}^T\textbf{A}) <= n < m$, so we can't take an inverse. This also makes sense intuitively; having more features than observations would usually mean that the features are linearly dependent, making it impossible to solve the problem.

## When $n = m$

We discussed this pretty extensively above, but TKTK can we say something about overfitting of some kind? Basically, given noise in observation, we wouldn't expect a perfect fit to be right

## When $n > m$, or more observations than features

Having more observations than features is the typical setting for linear regression. If the features are linearly independent, then $rank(\textbf{A}) = m$, so $rank(\textbf{A}^T\textbf{A}) = m$, which makes it invertible, so we can compute the solution $\textbf{x}^*$ above.

However, if the features are linearly dependent, we can't invert the matrix $\textbf{A}^T\textbf{A}$ to compute a solution. Because of numerical issues (more details on [wikipedia](https://en.wikipedia.org/wiki/Multicollinearity#Consequences)), it is also possible to encounter this kind of issue with columns that are collinear, but not entirely linearly dependent. Even when $\textbf{A}^T\textbf{A}$ can be inverted, collinear columns can make it difficult to interpret the weights $\textbf{x}^*$ TKTK

# References

- Inspiration and examples taken liberally from *Mathematics for Machine Learning* by Deisenroth, Faisal, and Ong.